In [1]:
#import dependencies
import os
import polars as pl
import pandas as pd

In [2]:
# Folder path
folder_path = r'C:\Users\hparnell\Desktop\MH10010\Resources'

In [3]:
def read_excel_files(folder_path):
    all_data = []
 
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
            file_path = os.path.join(folder_path, file_name)
            excel_data = pd.ExcelFile(file_path)
 
            for sheet_name in excel_data.sheet_names:
                sheet_df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
 
                # Remove rows with all NaN values (possible Headers and footers)
                sheet_df.dropna(how='all', inplace=True)
 
                # Convert problematic data types
                for col in sheet_df.columns:
                    if pd.api.types.is_datetime64_any_dtype(sheet_df[col]):
                        sheet_df[col] = sheet_df[col].astype(str)
                    elif pd.api.types.is_object_dtype(sheet_df[col]):
                        sheet_df[col] = sheet_df[col].astype(str)
 
                # Convert pandas DataFrame to polars DataFrame
                pl_df = pl.from_pandas(sheet_df)
 
                all_data.append(pl_df)
 
    # Concatenate all data into a single polars DataFrame
    combined_data = pl.concat(all_data)
 
    return combined_data

In [4]:
def convert_columns(df):
    for col in df.columns:
        try:
            df = df.with_column(pl.col(col).cast(pl.Float64, strict=False))
        except:
            try:
                df = df.with_column(pl.col(col).cast(pl.Int64, strict=False))
            except:
                df = df.with_column(pl.col(col).cast(pl.Utf8, strict=False))
    return df

In [ ]:
 # Read and combine data
combined_data = read_excel_files(folder_path)
 
# Convert columns to appropriate data types
cleaned_data = convert_columns(combined_data)
 
print(cleaned_data)